In [8]:
# -------------------------------
#     Loading Your Detections
# -------------------------------
#  Use the variables below to point the Toolbox scripts at your data.
#  Fill in the strings that are commented with # USER:  
#  then highlight a cell and hit Run Cell (play button above)

from common_python.load_detections import loadDetections
from common_python.filter_detections import filterDetections

# Set some variables
detection_file = 'acs_matched_detections_2012.csv'      # USER: And provide the filename here

version_id = '' #Version ID number [0 for initial(first) load]
time_interval = 60 #Time interval used to evaluate suspect detections (in minutes)


detection_radius = '' # Average Distance from receviers, in meters, tags can be detected
# There will be an opportunity later to change values for individual stations
# If entered: Value must be between 0 and 999 


# Create a suspect detections file?
SuspectDetections = True

# If you'd like to create a distance matrix for the stations in this detection file, 
# leave DistanceMatrix = True
DistanceMatrix = True
ReloadInputFile = True

table_names = loadDetections(detection_file, version_id, DistanceMatrix, 
                   ReloadInputFile, SuspectDetections, time_interval, detection_radius) # Wait for output below to say 'Loading complete.'

Verifying detection file: 'acs_matched_detections_2012.csv'... OK!
Loading records into acs_matched_detections_2012_v00, Please wait...
Creating geometry column for st_setsrid_4326, identified as WKB
Populating geometry column with ST_GeomFromEWKB
File loaded successfully! Detection Count: 90331
Processing detection summary on table acs_matched_detections_2012_v00 using the time interval: 60 minute(s)
Detection summary exported to: /home/vagrant/data/acs_matched_detections_2012_suspect_v00.csv
There are 1224 suspect detections
Station distance matrix exported to: /home/vagrant/data/acs_matched_detections_2012_distance_matrix_v00.csv
There are 171 station matrix pairs
Loading complete. Created/populated 3 tables


In [9]:
# -----------------------------------------------
#     Filtering Detections on Distance / Time
#
# -----------------------------------------------

# Tables created by this script and the script above are returned by 
# loadDetections() and listed in the 'table_names' variable.
# Types of table are Detection, station matrix, and suspect 


# If you have your own file of suspect detections, point to it here 
# and set OverrideSuspectDetectionFile to 'True':
OverrideSuspectDetectionFile = None
# Can replace the generated Suspect Detections file with your hand-edited file of suspect detections
SuspectFile = None # '/home/sandbox/data/sandbox/My_Suspect_Detections_File.csv' 

filterDetections(detection_file, version_id, SuspectFile, 
                     OverrideSuspectDetectionFile, DistanceMatrix, 
                     detection_radius, ReloadInputFile)  # Wait for Out[] to say 'Filtering complete.'

Using suspect detection file: /home/vagrant/data/acs_matched_detections_2012_suspect_v00.csv
Loading records into acs_matched_detections_2012_v00, Please wait...
Input detection file contains 90331 records
Loading records into suspects, Please wait...
Dropping existing table acs_matched_detections_2012_v01
1224 suspect detections removed
Total detections in output file: 89107
Detection file saved to: (/home/vagrant/data/acs_matched_detections_2012_v01.csv)
Station distance matrix exported to: /home/vagrant/data/acs_matched_detections_2012_distance_matrix_v01.csv
There are 169 station matrix pairs


'Filtering complete.'

In [7]:
# ---------------------
#     Interval Data 
# ---------------------

# Creates a compressed detections table and an interval data file
# Intervals are lengths of time in which a station detected an animal.
# Many consecutive detections of an animal are replaced by one interval

import common_python.interval_data_tool as inter_data

dist_matrix_file = "%s_distance_matrix_%s.%s" % (detection_file.split('.')[0], table_names['detections'].split('_')[-1], detection_file.split('.')[1])
# Or supply your own custom dist_matrix_filename here.
# dist_matrix_file = "my_custom_distance_matrix_filename.csv"

inter_data.intervalData(detection_file, dist_matrix_file)

Verifying Detection file: 'acs_matched_detections_2012.csv'... OK!
Verifying Distance Matrix file: 'acs_matched_detections_2012_distance_matrix_v00.csv'... OK!
Loading Detection file: 'acs_matched_detections_2012.csv'... OK!
Loading Distance Matrix file: 'acs_matched_detections_2012_distance_matrix_v00.csv'... OK!
Exporting data...
OK!
Compressed detections exported to: acs_matched_detections_2012_compressed_detections_v00.csv containing 7814 records.
Interval data exported to: acs_matched_detections_2012_interval_data_v00.csv containing 7814 records.


''

In [5]:
# ---------------------------
#    Distance Matrix Merge
# ---------------------------

# Merge two distance matrix files together. Useful for overriding distances 
# between stations that are not straight-line distances (ie river systems)

import common_python.dis_mtrx_merge as dis

# Merge two Distance Matrix files, letting the duplicate pair-distance values 

dist_matrix_file = "%s_distance_matrix_v00.%s" % tuple(detection_file.split('.')) # the full file that you had before
dist_matrix_file = 'acs_matched_detections_2012_distance_matrix_v00.csv'

# The file that has only the stations that require an overridden value for the distance between them 
# (in cases of stations with land or other obstacles between them which make a straight-line distance incorrect.
distance_real_input = 'acs_matched_detections_2012_distance_matrix_v01.csv'


dis.dis_mtx_merge('reqmerge', dist_matrix_file, distance_real_input)

Verifying distance matrix file: 'acs_matched_detections_2012_distance_matrix_v00.csv'... OK!
Verifying distance real file: 'acs_matched_detections_2012_distance_matrix_v01.csv'... OK!
There are 3 records updated in file acs_matched_detections_2012_distance_matrix_v00_merged.csv
Updates complete.


1